In [1]:
import pickle
#Run this with >bokeh serve --show TextAnalysis.ipynb
import pandas as pd
import numpy as np
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

from bokeh.layouts import layout,column,row
from bokeh.plotting import figure, show, output_file
import datashader as ds
import xarray
import time
import collections
import datashader.transfer_functions as tf


In [2]:
from bokeh.plotting import figure, output_notebook, show
output_notebook()

Loading BokehJS ...

In [3]:
table=pd.read_table('./TSOCS-failures-finalV.txt',header=None,
                    delimiter=',')
table.head()

,0,1,2,3,4,5,6,7
0,-3894.764404,-768.641602,1638.728271,945.853699,0.0,0.0,1685.511230,0.0
1,-2662.367676,-279.701660,-1910.987183,883.029297,0.0,0.0,129.657944,0.0
2,3317.957031,-291.576416,1442.459106,-1176.689087,0.0,0.0,-28.297564,0.0
3,3397.674316,811.834961,1718.503906,-1008.101746,0.0,0.0,-138.721893,0.0
4,-3366.577148,-1321.662231,-985.837708,-967.509155,0.0,0.0,19.367579,0.0


In [10]:
def base_plot(x_range,y_range,tools='pan,wheel_zoom,reset,hover'):
    p = figure(tools=tools, plot_width=600, plot_height=300,\
        x_range=x_range, y_range=y_range)   
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    p.add_tools(BoxZoomTool(match_aspect=True))
    return p

ndim=table.shape[1]
def dataframe_from_multiple_sequences(points,data):    
    ds_df=[]
    n_rows,n_cols=data.shape

    nans=np.zeros((n_rows,1))
    nans[:,0]=np.NaN
    data2=np.append(data,nans,1)
    
    for row in range(n_rows):
        df=pd.DataFrame(dict(x=np.arange(n_cols+1),y=data2[row,:]))
        ds_df.append(df)
        
    ds_df=pd.concat(ds_df)
    return ds_df
        

In [ ]:
from datashader.bokeh_ext import InteractiveImage
from bokeh.models import BoxZoomTool

matrix=table.as_matrix()
x_range=(0,len(matrix))
y_range=(np.min(matrix),np.max(matrix))

def create_image(x_range, y_range, w, h,matrix,cp,name):    
    print(matrix.shape)
    df = dataframe_from_multiple_sequences(np.arange(ndim),matrix)
    cvs = ds.Canvas(plot_height=400, plot_width=600)
    agg = cvs.line(df, 'x', 'y', ds.count())  
    img = tf.spread(tf.shade(agg, how='eq_hist',cmap=cp, name=name))
    return img

p1=base_plot(x_range,y_range)

w,h=800,600
InteractiveImage(p1,lambda x_range,y_range,w,h,: create_image( x_range,y_range,w,h,matrix,\
                                                             ['grey','black'],"Robotics"))
show(column(p1))

(73335, 8)
